In [2]:
import os
# os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

import cv2
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from PIL import Image
from skimage import io
import tensorflow as tf
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data
from torchvision import models
from torchvision.datasets import MNIST, FashionMNIST, CIFAR10
import torchvision.transforms as transforms
from tensorboardX import SummaryWriter



# data_path = "~/Datasets/iMet_Colelction_2019"
load_path = "~/Datasets/iMet_Colelction_2019/input/"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device", device)


# 複数GPU使用宣言
if device == 'cuda':
    net = torch.nn.DataParallel(net) # make parallel
    cudnn.benchmark = True


torch.manual_seed(1)

device cuda


In [ ]:
class iMetsDataset(data.Dataset):
    """Face Landmarks dataset."""
 
    def __init__(self, df, root_dir, transform=None):
        """
        Args:
            df (dataframe): ファイル名がindex、Nhot_LabelsカラムにNhot化したラベルを格納したDataframe
            root_dir (string): 対象の画像ファイルが入っているフォルダ
            transform (callable, optional): 施す変換
        """
        self.df = df
        self.root_dir = root_dir
        self.transform = transform
 
    def __len__(self):
        return len(self.df)
 
    def __getitem__(self, idx):
#         print(self.root_dir)
#         print(self.df.index)

        img_name = os.path.join(self.root_dir, self.df.index[idx])
        image = Image.fromarray(io.imread(img_name))
        label = self.df.iloc[idx].Nhot_Labels.astype('float32')
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
def Nhot_encoding(arr, l):
    """
    Nhotエンコーディングを行う

    Parameters
    ----------
    arr : ndarray
        ラベル
    l : int
        総ラベル数
    """
    ret = np.zeros(l,dtype='int')
    ret[arr] = 1
    return ret

In [ ]:
batch_size = 5
num_classes = 1103
epochs = 50

データ呼び出し

In [ ]:

label_name = pd.read_csv(load_path + 'labels.csv')
sample = pd.read_csv(load_path + 'sample_submission.csv')
train_df = pd.read_csv(load_path + 'train.csv')
train_size = len(train_df)

train_df["attribute_ids"] = train_df["attribute_ids"].apply(lambda x: np.array([int(s) for s in x.split(" ")]))
train_df["Nhot_Labels"] = train_df["attribute_ids"].apply(lambda x: Nhot_encoding(x,1103))
train_df["id"] = train_df["id"].apply(lambda x: x + ".png")
train_df = train_df.set_index('id')

データセットとデータローダの定義

In [ ]:
# データセット
train_ds = iMetsDataset(train_df,load_path+'train',
                            transform = transforms.Compose([
                            transforms.Resize((224,224)),
                            transforms.ToTensor(),
                            ])
                        )

# データローダ
dataloader_train = data.DataLoader(dataset=train_ds,batch_size=batch_size,shuffle=True)

In [ ]:
print(train_ds[0][0])

print(train_ds[0][1])